In [ ]:
#run
!pip install pygooglenews

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#run
!pip install snorkel
!pip install textblob
import io
import pandas as pd
from snorkel.labeling import LabelingFunction
import re
from snorkel.preprocess import preprocessor
from textblob import TextBlob
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.model import LabelModel
from snorkel.labeling import LFAnalysis
from snorkel.labeling import filter_unlabeled_dataframe
from snorkel.labeling import labeling_function
import spacy
from nltk.corpus import stopwords
import string
import nltk
import nltk.tokenize
punc = string.punctuation
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from tqdm import tqdm_notebook as tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#run
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/abcnews-date-text.csv')
df = df.rename(columns = {'headline_text': 'text'})
df['text'] = df['text'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1244184 entries, 0 to 1244183
Data columns (total 2 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   publish_date  1244184 non-null  int64 
 1   text          1244184 non-null  object
dtypes: int64(1), object(1)
memory usage: 19.0+ MB


In [ ]:
#run
n=50000

In [ ]:
df=df.sample(n)

In [ ]:
df.head()

,publish_date,text
400263,20080729,great lakes mayor quits
59421,20031205,violent protests rock port hedland detention c...
976616,20150904,mr wunungmurra state memorial service nhulunbu...
1117722,20180223,tasmanian shadow treasurer scott bacon rebuts
1033205,20160622,body found in water near newcastle believed to...


In [ ]:
df.shape

(50000, 2)

In [ ]:
#run
POSITIVE = 1
NEGATIVE = 0
ABSTAIN = -1
def keyword_lookup(x, keywords, label):
    if any(word in x.text.lower() for word in keywords):
        return label
    return ABSTAIN
def make_keyword_lf(keywords, label=POSITIVE):
    return LabelingFunction(
        name=f"keyword_{keywords[0]}",
        f=keyword_lookup,
        resources=dict(keywords=keywords, label=label))
keyword_positive = make_keyword_lf(keywords=['boosts', 'great', 'develops', 'promising', 'ambitious', 'delighted', 'record', 'win', 'breakthrough', 'recover', 'achievement', 'peace', 'party', 'hope', 'flourish', 'respect', 'partnership', 'champion', 'positive', 'happy', 'bright', 'confident', 'encouraged', 'perfect', 'complete', 'assured' ])

keyword_negative = make_keyword_lf(keywords=['war','solidiers', 'turmoil', 'injur','trouble', 'aggressive', 'killed', 'coup', 'evasion', 'strike', 'troops', 'dismisses', 'attacks', 'defeat', 'damage', 'dishonest', 'dead', 'fear', 'foul', 'fails', 'hostile', 'cuts', 'accusations', 'victims',  'death', 'unrest', 'fraud', 'dispute', 'destruction', 'battle', 'unhappy', 'bad', 'alarming', 'angry', 'anxious', 'dirty', 'pain', 'poison', 'unfair', 'unhealthy'
                                              ], label=NEGATIVE)

In [ ]:
#run
@preprocessor(memoize=True)
def textblob_sentiment(x):
    scores = TextBlob(x.text)
    x.polarity = scores.sentiment.polarity
    x.subjectivity = scores.sentiment.subjectivity
    return x
@labeling_function(pre=[textblob_sentiment])
def textblob_polarity(x):
    return POSITIVE if x.polarity > 0.6 else ABSTAIN
@labeling_function(pre=[textblob_sentiment])
def textblob_subjectivity(x):
    return POSITIVE if x.subjectivity >= 0.5 else ABSTAIN

In [ ]:
lfs = [keyword_positive, keyword_negative, textblob_polarity, textblob_subjectivity ]
applier = PandasLFApplier(lfs=lfs)
L_snorkel = applier.apply(df=df)
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_snorkel)
df["label"] = label_model.predict(L=L_snorkel)

100%|██████████| 100/100 [00:00<00:00, 1088.04epoch/s]


In [ ]:
df= df.loc[df.label.isin([0,1]), :]
df['label'].value_counts()

1    9521
0    4469
Name: label, dtype: int64

In [ ]:
df.to_csv("data_ABC_NEWS_LABELLED.csv", index=False)

In [ ]:
data=pd.read_csv('data_ABC_NEWS_LABELLED.csv')

In [ ]:
data = df.copy()
def preparation_text_data(data):
    """
    This pipeline prepares the text data, conducting the following steps:
    1) Tokenization
    2) Lemmatization
    4) Removal of stopwords
    5) Removal of punctuation
    """
    # initialize spacy object
    nlp = spacy.load('en_core_web_sm')
    # select raw text
    raw_text = data.text.values.tolist()
    # tokenize
    tokenized_text = [[nlp(i.lower().strip())] for i in tqdm(raw_text)]
    #define the punctuations and stop words
    punc = string.punctuation
    stop_words = set(stopwords.words('english'))
    #lemmatize, remove stopwords and punctuationd
    corpus = []
    for doc in tqdm(tokenized_text):
        corpus.append([word.lemma_ for word in doc[0] if (word.lemma_ not in stop_words and word.lemma_ not in punc)])
    # add prepared data to df
    data["text"] = corpus
    return data
#apply the data preprocessing function
data =  preparation_text_data(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/13990 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/13990 [00:00<?, ?it/s]

In [ ]:
# data.to_csv('data_ABC_NEWS_LABELLED_1.csv',index=False)

In [ ]:
# run
# data=pd.read_csv('data_ABC_NEWS_LABELLED_1.csv')

In [ ]:
#run
def text_representation(data):
  tfidf_vect = TfidfVectorizer()
  data['text'] = data['text'].apply(lambda text: " ".join(set(text)))
  X_tfidf = tfidf_vect.fit_transform(data['text'])
  print(X_tfidf.shape)
  print(tfidf_vect.get_feature_names())
  X_tfidf = pd.DataFrame(X_tfidf.toarray())
  return X_tfidf
#apply the TFIDV function
X_tfidf = text_representation(data)

(13990, 11967)
['000', '04', '0605', '08', '080808', '0910', '10', '100', '1000', '10000', '100000', '100k', '100pc', '100th', '103', '1057', '1057s', '106', '10k', '10pc', '10th', '11', '110k', '110th', '1140', '118b', '12', '120', '12000', '1220', '123456', '125', '125cc', '126000', '12s', '12yo', '13', '130', '130000', '134', '135', '13th', '13yo', '14', '140', '14000', '14th', '14yo', '15', '1500', '15000', '154', '15k', '16', '160', '16000', '1607', '16th', '16yo', '17', '170', '175', '17500', '17c', '17th', '17yos', '18', '180', '185', '188', '18th', '19', '1918', '1920', '1942', '197', '1971', '1980', '1986', '1988', '1991', '1995', '1996', '1997', '19th', '1b', '20', '200', '20000', '2002', '2003', '2004', '2006', '2007', '200708', '2008', '2009', '200k', '2010', '2010s', '2011', '20111', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2050', '20th', '21', '2100', '2107', '215k', '217', '21st', '22', '2200', '224', '229', '22k', '22yo', '23', '2

In [ ]:
data.shape

(13990, 3)

In [ ]:
#Filtering out unlabeled data points
data= data.loc[data.label.isin([0,1]), :]
#find the label counts
data['label'].value_counts()

1    9521
0    4469
Name: label, dtype: int64

In [ ]:
i=9521-4469
data.drop(data[data['label'] ==1].sample(frac=.2).index,inplace=True)

In [ ]:
data= data.loc[data.label.isin([0,1]), :]
#find the label counts
data['label'].value_counts()

1    6855
0    4469
Name: label, dtype: int64

In [ ]:
# #run
# def text_representation(data):
#   tfidf_vect = TfidfVectorizer()
#   data['text'] = data['text'].apply(lambda text: " ".join(set(text)))
#   X_tfidf = tfidf_vect.fit_transform(data['text'])
#   print(X_tfidf.shape)
#   print(tfidf_vect.get_feature_names())
#   X_tfidf = pd.DataFrame(X_tfidf.toarray())
#   return X_tfidf
# #apply the TFIDV function
# X_tfidf = text_representation(data)

ValueError: ignored

In [ ]:
#run
X= X_tfidf
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#fit Log Regression Model
clf= LogisticRegression()
clf.fit(X_train,y_train)
clf.score(X_test,y_test)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.81      0.85      1483
           1       0.91      0.96      0.94      3134

    accuracy                           0.91      4617
   macro avg       0.91      0.88      0.89      4617
weighted avg       0.91      0.91      0.91      4617



In [ ]:
#run
##store headlines and labels in respective lists
text = list(data['text'])
labels = list(data['label'])
##sentences
L=(int)(0.7*n)
training_text = text[0:L]
testing_text = text[L:]
##labels
training_labels = labels[0:L]
testing_labels = labels[L:]

In [ ]:
#run
from pygooglenews import GoogleNews

In [ ]:
#run
#preprocess
tokenizer = Tokenizer(num_words=10000, oov_token= "<OOV>")
tokenizer.fit_on_texts(training_text)
word_index = tokenizer.word_index
training_sequences = tokenizer.texts_to_sequences(training_text)
training_padded = pad_sequences(training_sequences, maxlen=120, padding='post', truncating='post')
testing_sequences = tokenizer.texts_to_sequences(testing_text)
testing_padded = pad_sequences(testing_sequences, maxlen=120, padding='post', truncating='post')
# convert lists into numpy arrays to make it work with TensorFlow
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [ ]:
#run
from tensorflow import keras

In [ ]:
model = keras.Sequential([
    keras.layers.Embedding(10000, 16, input_length=120),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
##compile the model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 120, 16)           160000    
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 24)                408       
                                                                 
 dense_3 (Dense)             (None, 1)                 25        
                                                                 
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 10
history = model.fit(training_padded,
                    training_labels,
                    epochs=num_epochs,
                    validation_data=(testing_padded, testing_labels),
                    verbose=2)

Epoch 1/10
354/354 - 2s - loss: 0.6725 - accuracy: 0.6027 - 2s/epoch - 6ms/step
Epoch 2/10
354/354 - 1s - loss: 0.6617 - accuracy: 0.6054 - 1s/epoch - 4ms/step
Epoch 3/10
354/354 - 1s - loss: 0.5918 - accuracy: 0.6522 - 1s/epoch - 4ms/step
Epoch 4/10
354/354 - 1s - loss: 0.3708 - accuracy: 0.8941 - 1s/epoch - 4ms/step
Epoch 5/10
354/354 - 1s - loss: 0.2160 - accuracy: 0.9420 - 1s/epoch - 4ms/step
Epoch 6/10
354/354 - 1s - loss: 0.1536 - accuracy: 0.9535 - 1s/epoch - 4ms/step
Epoch 7/10
354/354 - 1s - loss: 0.1230 - accuracy: 0.9618 - 1s/epoch - 4ms/step
Epoch 8/10
354/354 - 1s - loss: 0.1027 - accuracy: 0.9683 - 1s/epoch - 4ms/step
Epoch 9/10
354/354 - 1s - loss: 0.0871 - accuracy: 0.9739 - 1s/epoch - 4ms/step
Epoch 10/10
354/354 - 1s - loss: 0.0762 - accuracy: 0.9767 - 1s/epoch - 4ms/step


In [ ]:
#run
def newsSentiment(x):
    x=x[0][0]
    if(x<0.2):
        return 'Highly negative'
    elif(x<0.4):
        return 'Negative'
    elif(x<0.6):
        return 'Neutral'
    elif(x<0.8):
        return 'Positive'
    else:
        return 'Highly Positive'

In [ ]:
model.save('sentiment_model')

In [ ]:
#run
new_model = keras.models.load_model('sentiment_model')

In [ ]:
#run
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 120, 16)           160000    
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 24)                408       
                                                                 
 dense_3 (Dense)             (None, 1)                 25        
                                                                 
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#run
new_headline = ["The US imposes sanctions on Rassia because of the Ukranian war"]
##prepare the sequences of the sentences in question
sequences = tokenizer.texts_to_sequences(new_headline)
padded_seqs = pad_sequences(sequences, maxlen=120, padding='post', truncating='post')
print(new_headline,new_model.predict(padded_seqs))

1/1 [==============================] - 0s 24ms/step
['The US imposes sanctions on Rassia because of the Ukranian war'] [[0.01764822]]


In [ ]:
#run
data.head()

In [ ]:
#run
from pygooglenews import GoogleNews

gn = GoogleNews()
newsList=[]
s = gn.search('Hubspot')
count=0
for entry in s["entries"]:
  count+=1
  print(count,entry["link"])
  newsList.append(entry['link'])
  if count==10:
    break

1 https://news.google.com/__i/rss/rd/articles/CBMiowFodHRwczovL3d3dy5wcm5ld3N3aXJlLmNvbS9uZXdzLXJlbGVhc2VzL2RlbWFuZGJhc2UtZGVsaXZlcnMtZnVsbHktaW50ZWdyYXRlZC1hY2NvdW50LWJhc2VkLW1hcmtldGluZy1hYm0tZm9yLWh1YnNwb3QtY3JtLWFuZC1taWNyb3NvZnQtZHluYW1pY3MtMzY1LTMwMTY3OTc3OC5odG1s0gEA?oc=5
2 https://news.google.com/__i/rss/rd/articles/CBMinwFodHRwczovL3d3dy5idXNpbmVzc3dpcmUuY29tL25ld3MvaG9tZS8yMDIyMTExNTAwNTM3MS9lbi9IdWJTcG90LUFubm91bmNlcy1HZW5lcmFsLUF2YWlsYWJpbGl0eS1vZi1XaGF0c0FwcC1JbnRlZ3JhdGlvbi10by1IZWxwLUN1c3RvbWVycy1Gb3JtLURlZXAtQ29ubmVjdGlvbnPSAQA?oc=5
3 https://news.google.com/__i/rss/rd/articles/CBMiZWh0dHBzOi8vd3d3Lm5hc2RhcS5jb20vYXJ0aWNsZXMvaG93LW11Y2gtdXBzaWRlLWlzLWxlZnQtaW4taHVic3BvdC1odWJzLXdhbGwtc3RyZWV0LWFuYWx5c3RzLXRoaW5rLTM40gEA?oc=5
4 https://news.google.com/__i/rss/rd/articles/CBMirgFodHRwczovL3d3dy5wcm5ld3N3aXJlLmNvbS9uZXdzLXJlbGVhc2VzL2NsaWNrdXAtYW5kLWh1YnNwb3QtYW5ub3VuY2Utc3RyYXRlZ2ljLXBhcnRuZXJzaGlwLWVtcG93ZXJpbmctYnVzaW5lc3Nlcy10by1pbmNyZWFzZS1wcm9kdWN0aXZpdHktYW5kLXJldGF

In [ ]:
#run
!pip install newspaper3k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#run
import newspaper
import json

for i in newsList:
  article = newspaper.Article(url=i, language='en')
  article.download()
  article.parse()
  print(article.text)

New integrations mean customers of all major CRMs can now take full advantage of Demandbase's capabilities, with bidirectional sync and more automated go-to-market workflows

SAN FRANCISCO, Nov. 16, 2022 /PRNewswire/ -- Demandbase , the Smarter GTM™ company for B2B brands, today announced native, bidirectional integrations with Microsoft Dynamics 365 and HubSpot CRM. This integration enables customers to use their CRM of choice to hit their revenue goals with zero waste, by identifying, engaging, and focusing their resources on accounts and prospects who are most likely to buy.

Dynamics 365 and HubSpot customers will now be able to push their CRM data into Demandbase One to inform and automate workflows, such as Lead-to-Account mapping, and tracking and taking action on engagement. Powered by a combination of the customer's data, Demandbase's proprietary data, and AI — what we call Account Intelligence — they will also be able to see and act on Demandbase intent and engagement data wi

In [ ]:
#run
!pip install datasets
!pip install transformers
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#run
import datasets  #(1.13.3)
import pandas as pd  #(0.25.1)
import numpy  #(1.16.4)

## for preprocessing
import re
import nltk  #(3.4.5)
import contractions  #(0.0.18)
## for textrank
import difflib
## for bart
import transformers

In [ ]:
import pandas as pd
import datasets
dataset = datasets.load_dataset("cnn_dailymail", '3.0.0')

lst_dics = [dic for dic in dataset["train"]]
dtf = pd.DataFrame(lst_dics).rename(columns={"article":"text", "highlights":"y"})[["text","y"]].head(12000)
dtf.to_csv("data_summary.csv", index=False)
dtf.head()

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

,text,y
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa..."
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non..."
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical..."


In [ ]:
#run
dtf=pd.read_csv('data_summary.csv')

In [ ]:
#run
dtf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    12000 non-null  object
 1   y       12000 non-null  object
dtypes: object(2)
memory usage: 187.6+ KB


In [ ]:
#run
# check
i = 1
print("--- Full text ---")
print(dtf["text"][i])
print(" ")
print("--- Summary ---")
print(dtf["y"][i])

--- Full text ---
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won't do what they're told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less li

In [ ]:

def bart(corpus, max_len):
    nlp = transformers.pipeline("summarization")
    lst_summaries = [nlp(txt,
                         max_length=200
                         )[0]["summary_text"].replace(" .", ".")
                     for txt in corpus]
    return lst_summaries

## Apply the function to corpus
predicted = bart(corpus=dtf["text"], max_len=2000)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


KeyboardInterrupt: ignored

In [ ]:
text="Shares of Cisco Systems Inc. CSCO, -0.66% slipped 0.66% to $48.08 Monday, on what proved to be an all-around poor trading session for the stock market, with the S&P 500 Index SPX, -1.54% falling 1.54% to 3,963.94 and Dow Jones Industrial Average DJIA, -1.45% falling 1.45% to 33,849.46. This was the stock's second consecutive day of losses. Cisco Systems Inc. closed $16.21 short of its 52-week high ($64.29), which the company reached on December 29th.The stock demonstrated a mixed performance when compared to some of its competitors Monday, as Microsoft Corp. MSFT, -2.32% fell 2.32% to $241.76, Amazon.com Inc. AMZN, +0.58% rose 0.58% to $93.95, and Broadcom Inc. AVGO, -1.45% fell 1.45% to $522.21. Trading volume (14.9 M) remained 5.2 million below its 50-day average volume of 20.2 M. Editor's Note: This story was auto-generated by Automated Insights, an automation technology provider, using data from Dow Jones and FactSet. See our market data terms of use."
to_tokenize = f.readlines()
# Initialize the HuggingFace summarization pipeline
summarizer = pipeline("summarization")
summarized = summarizer(to_tokenize, min_length=75, max_length=300)


# # Print summarized text
print(summarized)

In [ ]:
with open('model_pkl', 'wb') as files:
    pickle.dump(model, files)